In [1]:
# 先不取平均 看一看能否说明问题
import numpy as np
import sys
sys.path.append(r'C:\Users\zhang\1_jupyter_code\1_Functions')
import matplotlib.pyplot as plt
from cal_ppp_dis import cal_ppp_distance as cal_dis
from pyscal3 import System
from pyscal3 import Atoms

In [2]:
# 读取xyz数据
file_path = r'C:\Users\zhang\Desktop\md\las_growth\las_13-23k\19k.xyz'
save_path = r'C:\Users\zhang\Desktop\xyz_q4.lammpstrj'
start_frame = 0
end_frame = 49
step_frame = 1
###########

# 要分析的元素
ana_element = ['Si','O','Al']
# 画出分布的元素
draw_element = ['O']
set_cutoff = 4.0

In [3]:
def read_xyz_frames(file_path):
    frames = []
    current_frame = None

    with open(file_path, 'r') as file:
        lines = file.readlines()
        line_index = 0

        while line_index < len(lines):
            line = lines[line_index].strip()

            if line.isdigit():
                # 读取原子数
                if current_frame:
                    frames.append(current_frame)
                num_atoms = int(line)
                current_frame = {"num_atoms": num_atoms, "atoms": []}
                line_index += 2  # 跳过标题行 "Atoms"
            else:
                # 读取原子坐标
                if current_frame:
                    atom_data = line.split()
                    if len(atom_data) == 4:
                        element, x, y, z = atom_data
                        current_frame["atoms"].append({
                            "element": element,
                            "x": float(x),
                            "y": float(y),
                            "z": float(z)
                        })
                line_index += 1

        if current_frame:
            frames.append(current_frame)

    return frames

# 使用之前的frames列表
frames = read_xyz_frames(file_path)

# 找出最大的x、y和z值
max_x = max([max(frame["atoms"], key=lambda atom: atom["x"])["x"] for frame in frames])
max_y = max([max(frame["atoms"], key=lambda atom: atom["y"])["y"] for frame in frames])
max_z = max([max(frame["atoms"], key=lambda atom: atom["z"])["z"] for frame in frames])

print("Maximum X value:", max_x)
print("Maximum Y value:", max_y)
print("Maximum Z value:", max_z)


Maximum X value: 84.713001
Maximum Y value: 55.321148
Maximum Z value: 11.50816


In [4]:
box_size_old = [max_x+0.01,max_y+0.01,max_z+0.01]
print(box_size_old)
print(frames[0]['atoms'][0])

[84.72300100000001, 55.331148, 11.51816]
{'element': 'O', 'x': 59.619703, 'y': 30.15371, 'z': 1.33692}


In [5]:
def cal_bop(atom_list,ana_element,draw_element):
    sys = System()
    sys.box = [[max_x+0.01,0,0],[0,max_y+0.01,0],[0,0,max_z+0.01]]
    # 将list转为字典 取出我们要分析的元素 
    ana_atom_list = []
    atom_xyz = []
    for i in data_list:
        if i[0] in ana_element:
            atom_xyz.append(i[1:])
            ana_atom_list.append(i)
    #print(np.array(atom_xyz).shape)
    # 转换元素格式
    adict = {"positions": np.array(atom_xyz, dtype=object)}
    atoms = Atoms(adict)
    # 实例化
    sys.atoms = atoms
    # 计算
    sys.find.neighbors(method='voronoi')
    coordination = [len(sys.atoms.neighbors.index[x]) for x in range(sys.natoms)]
    # 计算q4和q6 然后采用均值的方法
    q4_q6_list = sys.calculate.steinhardt_parameter([4,6], averaged=True)
    # 分配值
    q4 = q4_q6_list[0]
    q6 = q4_q6_list[1]
    results_xyz_q4_q6 = []
    for i,i_atom in enumerate(ana_atom_list):
        if i_atom[0] in draw_element:
            results_xyz_q4_q6.append([i_atom[1],i_atom[2],i_atom[3],q4[i],q6[i]])
    return results_xyz_q4_q6

In [6]:
head = 'ITEM: TIMESTEP\n{}\nITEM: NUMBER OF ATOMS\n{}\nITEM: BOX BOUNDS xy xz \
pp pp pp\n0 {}\n0 {} \n0 {}\nITEM: ATOMS id x y z vx vy vz '.format(22,10,22,22,22)
print(head)
row = '{} {} {} {} {} 0 0'.format(0,1, 1,2,1,0.5)
print(row)

ITEM: TIMESTEP
22
ITEM: NUMBER OF ATOMS
10
ITEM: BOX BOUNDS xy xz pp pp pp
0 22
0 22 
0 22
ITEM: ATOMS id x y z vx vy vz 
0 1 1 2 1 0 0


In [7]:
def write_xyz_q4(save_path, result_xyz_q4_q6,i_frame,atom_num):
    # 打开文件以追加内容
    box_x,box_y,box_z = 84.713001, 55.321148, 11.50816
    head = head = 'ITEM: TIMESTEP\n{}\nITEM: NUMBER OF ATOMS\n{}\nITEM: BOX BOUNDS xy xz \
pp pp pp\n0 {}\n0 {} \n0 {}\nITEM: ATOMS id x y z vx vy vz \n'.format(i_frame,atom_num,box_x,box_y,box_z)
    if i_frame==0:
        w_type = 'w'
    else:
        w_type = 'a'
    with open(save_path, w_type) as file:
        # 追加内容
        file.write(head)
        for i in range(len(result_xyz_q4_q6)):
            x,y,z,vx = result_xyz_q4_q6[i][0],result_xyz_q4_q6[i][1],result_xyz_q4_q6[i][2],result_xyz_q4_q6[i][3]
            row = '{} {} {} {} {} 0 0\n'.format(i,x,y,z,vx*100)
            file.write(row)

# 开始多帧计算分析

In [8]:
for i_frame in range(start_frame,end_frame,step_frame):
    data_list = []
    for i in frames[i_frame]['atoms']:
        data_list.append([i['element'],i['x'],i['y'],i['z']])
    result_xyz_q4_q6 = cal_bop(data_list,ana_element,draw_element)
    # 得到结果了 然后把结果存下来
    write_xyz_q4(save_path,result_xyz_q4_q6,i_frame,2292)

D:\python_install\lib\site-packages\pyscal3\atoms.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dict.__setitem__(self, key, np.array(val))
